In [ ]:
pip install pyodbc

In [ ]:
import pyodbc

def connect_db():
    server = 'your_server_name'
    database = 'your_database_name'
    username = 'your_username'
    password = 'your_password'
    driver= '{ODBC Driver 18 for SQL Server}'  # Update driver if necessary
    connection_string = f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password}'
    connection = pyodbc.connect(connection_string)
    return connection

def disconnect_db(connection):
    connection.close()


In [ ]:
def create_book(connection, title, author, genre, publication_year):
    cursor = connection.cursor()
    sql = "INSERT INTO Books (Title, Author, Genre, PublicationYear) VALUES (?, ?, ?, ?);"
    cursor.execute(sql, (title, author, genre, publication_year))
    connection.commit()
    cursor.close()


In [ ]:
def read_books(connection, criteria=None):
    cursor = connection.cursor()
    # Basic query
    sql = "SELECT * FROM Books"

    # Apply criteria if searching or filtering
    if criteria:
        sql += f" WHERE Title LIKE ? OR Author LIKE ?"
        cursor.execute(sql, (f"%{criteria}%", f"%{criteria}%"))
    else:
        cursor.execute(sql)

    books = cursor.fetchall()
    for book in books:
        print(f"Title: {book.Title}, Author: {book.Author}, Genre: {book.Genre}")

    cursor.close()



In [ ]:
def update_book(connection, book_id, new_title=None, new_author=None, new_genre=None, new_year=None):
    cursor = connection.cursor()
    update_sql = "UPDATE Books SET "
    update_fields = []
    values = []

    if new_title:
        update_fields.append("Title = ?")
        values.append(new_title)
    if new_author:
        update_fields.append("Author = ?")
        values.append(new_author)
    if new_genre:
        update_fields.append("Genre = ?")
        values.append(new_genre)
    if new_year:
        update_fields.append("PublicationYear = ?")
        values.append(new_year)

    update_sql += ", ".join(update_fields)
    update_sql += " WHERE BookID = ?"
    values.append(book_id)

    cursor.execute(update_sql, values)
    connection.commit()
    print(f"Book with ID {book_id} updated successfully.")
    cursor.close()

In [ ]:
def delete_book(connection, book_id):
    cursor = connection.cursor()
    sql = "DELETE FROM Books WHERE BookID = ?;"
    cursor.execute(sql, (book_id,))
    connection.commit()
    cursor.close()


In [ ]:
import pyodbc

# Assuming CRUD functions are imported from the igottabook database module
# from igottabook_database import connect_db, create_book, read_books, update_book, delete_book

# Connect to the database
conn = None
try:
    conn = connect_db()  # Establish connection

    # 1. Test Create (Insert)
    print("Testing create_book...")
    create_book(conn, "Test Book", "Test Author", "Fiction", 2023)
    print("Book created.")

    # 2. Test Read (Query)
    print("Testing read_books...")
    read_books(conn)  # Should list books including "Test Book" if it was created successfully

    # 3. Test Update (Modify)
    print("Testing update_book...")
    update_book(conn, book_id=1, new_title="Updated Test Book")  # Assuming book_id 1 for testing purposes
    print("Book updated.")

    # 4. Test Delete (Remove)
    print("Testing delete_book...")
    delete_book(conn, 1)  # Again assuming book_id 1
    print("Book deleted.")

except Exception as e:
    print("Error:", e)

finally:
    # Close the connection
    if conn:
        disconnect_db(conn)


Error: ('01000', "[01000] [unixODBC][Driver Manager]Can't open lib 'ODBC Driver 18 for SQL Server' : file not found (0) (SQLDriverConnect)")
